## Library and data import

In [1]:
library(rstatix)
library(stringr)
library(tidyverse)
library(RColorBrewer)
library(dplyr)
library(tidyr)
library(tibble)
library(readr)
library(ggplot2)
library(fs)
library(gt)
library(readr)


Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ purrr     1.1.0
✔ forcats   1.0.1     ✔ readr     2.1.5
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks rstatix::filter(), stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


STRs data

In [2]:
df_strs <- read_tsv("../samples/gp_global/merged/outliers_annotated_complete.tsv")

head(df_strs)
colnames(df_strs)

Rows: 3333995 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (9): STRs_ID, region, gene_id, gene_name, gene_chrom, annotation, sampl...
dbl (10): priority, gene_start, gene_end, start, end, allele1_est, allele2_e...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


STRs_ID,region,gene_id,priority,gene_name,gene_chrom,gene_start,gene_end,annotation,sample_id,chrom,start,end,repeat_unit,allele1_est,allele2_est,depth,p,p_adj
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr18:35064000:A:34,intron,ENSG00000166974,5,MAPRE2,chr18,34976928,35143470,intron:MAPRE2,BEL-1012-1,18,35064000,35064033,A,-9,155.68,33,1.7e-157,1.5e-152
chr1:69249900:T:2,intergenic,intergenic,6,.,.,-1,-1,intergenic,BEL-1012-1,1,69249900,69249901,T,0,112.16,32,8.8e-143,3.9e-138
chr4:93128027:A:33,intron,ENSG00000152208,5,GRID2,chr4,92303966,93810157,intron:GRID2,BEL-1012-1,4,93128027,93128059,A,0,151.43,37,2.5e-127,2.0e-123
chr1:108526008:A:2,intergenic,intergenic,6,.,.,-1,-1,intergenic,BEL-1012-1,1,108526008,108526009,A,0,77.30,38,1.9e-88,1.6e-85
chr12:46783052:GT:4,intron,ENSG00000139209,5,SLC38A4,chr12,46764761,46832408,intron:SLC38A4,BEL-1012-1,12,46783052,46783059,GT,0,19.24,33,1.1e-46,9.8e-45
chr11:14508561:A:17,intron,ENSG00000129084,5,PSMA1,chr11,14504874,14643635,intron:PSMA1,BEL-1012-1,11,14508561,14508577,A,0,88.74,38,1.1e-05,9.6e-05


[1] "STRs_ID"     "region"      "gene_id"     "priority"    "gene_name"  
 [6] "gene_chrom"  "gene_start"  "gene_end"    "annotation"  "sample_id"  
[11] "chrom"       "start"       "end"         "repeat_unit" "allele1_est"
[16] "allele2_est" "depth"       "p"           "p_adj"

Import list of genes expressed in lung tissue (COVID-19), identified via sc-RNAseq

In [5]:
# 1. Load Gene Data
# We do NOT remove gene duplicates here, as we want all entries
df_genes <- read_csv("merged_scovid.csv") %>%
  select(-Accession) # Remove only irrelevant technical columns

# 2. DEBUG: Initial Counts
cat("\n--- DEBUG: Initial Counts ---\n")
n_total_strs <- nrow(df_strs) # Your original STR dataframe (multiple patients)
n_gene_entries <- nrow(df_genes) # Entries in the gene list (may have repeated genes per tissue)

cat(sprintf("Total STRs (observations): %d\n", n_total_strs))
cat(sprintf("Total Gene Entries (context rows): %d\n", n_gene_entries))

# 3. Full Merge (Many-to-Many)
df_filtered <- df_strs %>%
  inner_join(df_genes, by = c("gene_name" = "Gene Symbol"), relationship = "many-to-many")

# 4. DEBUG: Merge Results
n_final_rows <- nrow(df_filtered)

# Calculating how many unique STR observations (Locus + Patient) were actually kept
n_unique_strs <- n_distinct(df_filtered$STRs_ID, df_filtered$sample_id) 

cat("\n--- DEBUG: Merge Results ---\n")
cat(sprintf("Final Rows (Expanded): %d\n", n_final_rows))
cat(sprintf("Unique STR Observations covered: %d\n", n_unique_strs))
cat("Note: The row count reflects the expansion where each STR is listed for every tissue/dataset context of its gene.\n")

# 5. Preview
cat("\n--- Preview ---\n")
print(head(df_filtered))

# Save results
output_dir <- "./results"
  if (!dir.exists(output_dir)) dir.create(output_dir)
  write_csv(df_filtered, file.path(output_dir, "STRs_COVID_Expanded_Context.csv"))


Rows: 2154 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): Gene Symbol, Ensembl ID, Dataset, Accession, Tissue, origem
dbl (2): Pvalue, LogFC

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



--- DEBUG: Initial Counts ---
Total STRs (observations): 3333995
Total Gene Entries (context rows): 2154

--- DEBUG: Merge Results ---
Final Rows (Expanded): 341923
Unique STR Observations covered: 144796
Note: The row count reflects the expansion where each STR is listed for every tissue/dataset context of its gene.

--- Preview ---
# A tibble: 6 × 25
  STRs_ID       region gene_id priority gene_name gene_chrom gene_start gene_end
  <chr>         <chr>  <chr>      <dbl> <chr>     <chr>           <dbl>    <dbl>
1 chr17:667995… intron ENSG00…        5 PRKCA     chr17        66302613   6.68e7
2 chr17:667995… intron ENSG00…        5 PRKCA     chr17        66302613   6.68e7
3 chr7:1820753… intron ENSG00…        5 HDAC9     chr7         18086949   1.90e7
4 chr18:687319… intron ENSG00…        5 CCDC102B  chr18        68715209   6.91e7
5 chr3:1058690… intron ENSG00…        5 CBLB      chr3        105655461   1.06e8
6 chr6:1292224… intron ENSG00…        5 LAMA2     chr6        128883141   1.3

Import association between samples and groups

In [6]:
# Load and Clean Group Data
df_groups <- read_csv("grupos.csv") %>%
  # Remove the ".bam" extension from the sample column to match sample_id
  mutate(sample_clean = str_remove(sample, "\\.bam$")) %>%
  select(sample_clean, group) # Keep only the clean ID and the group

# DEBUG: Check ID Match before joining
cat("\n--- DEBUG: Checking ID Match ---\n")
ids_filtered <- unique(df_filtered$sample_id)
ids_groups <- unique(df_groups$sample_clean)

# How many IDs from df_filtered are present in the groups file?
ids_match <- intersect(ids_filtered, ids_groups)
n_missing <- length(setdiff(ids_filtered, ids_groups))

cat(sprintf("Total unique samples in STR data: %d\n", length(ids_filtered)))
cat(sprintf("Total samples with group info found: %d\n", length(ids_match)))

if (n_missing > 0) {
  cat(sprintf("WARNING: %d samples in STR data have NO group info!\n", n_missing))
  # Optional: Check which ones are missing
} else {
  cat("SUCCESS: All samples have group info.\n")
}

# 4. Integrate (Left Join to keep STR data even if group info is missing)
df_filtered_grouped <- df_filtered %>%
  left_join(df_groups, by = c("sample_id" = "sample_clean"))

# 5. Preview Result
cat("\n--- Preview with Groups ---\n")
df_filtered_grouped %>%
  select(sample_id, group, everything()) %>% # Move group to the front for easy viewing
  head() %>%
  print()

# 6. Save enriched CSV
output_path <- "STRs_COVID_Grouped.csv"
write_csv(df_filtered_grouped, output_path)
print("Total number of observations:")
nrow(df_filtered_grouped)
cat(sprintf("\nFile saved to: %s\n", output_path))


Rows: 168 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): sample, group

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



--- DEBUG: Checking ID Match ---
Total unique samples in STR data: 168
Total samples with group info found: 168
SUCCESS: All samples have group info.

--- Preview with Groups ---
# A tibble: 6 × 26
  sample_id   group STRs_ID         region gene_id priority gene_name gene_chrom
  <chr>       <chr> <chr>           <chr>  <chr>      <dbl> <chr>     <chr>     
1 GIG-RS-15-1 case  chr17:66799505… intron ENSG00…        5 PRKCA     chr17     
2 GIG-RS-15-1 case  chr17:66799505… intron ENSG00…        5 PRKCA     chr17     
3 GIG-RS-15-1 case  chr7:18207539:… intron ENSG00…        5 HDAC9     chr7      
4 GIG-RS-15-1 case  chr18:68731924… intron ENSG00…        5 CCDC102B  chr18     
5 GIG-RS-15-1 case  chr3:105869062… intron ENSG00…        5 CBLB      chr3      
6 GIG-RS-15-1 case  chr6:129222479… intron ENSG00…        5 LAMA2     chr6      
# ℹ 18 more variables: gene_start <dbl>, gene_end <dbl>, annotation <chr>,
#   chrom <chr>, start <dbl>, end <dbl>, repeat_unit <chr>, allele1_est <dbl>,

[1] 341923


File saved to: STRs_COVID_Grouped.csv


# Statistical analysis

1. Calculate Descriptive Statistics by GROUP

In [35]:
# Ensure the results directory exists
output_dir <- "./results"
if (!dir_exists(output_dir)) dir_create(output_dir)

# 1. Descriptive analysis by group
allele_stats_group <- df_filtered_grouped %>%
  group_by(group) %>%
  summarise(
    n_obs = n(),
    across(c(allele1_est, allele2_est), 
           list(mean = ~mean(.x, na.rm = TRUE), 
                median = ~median(.x, na.rm = TRUE), 
                sd = ~sd(.x, na.rm = TRUE),
                iqr = ~IQR(.x, na.rm = TRUE)),
           .names = "{.col}_{.fn}")
  ) %>%
  pivot_longer(cols = -group, names_to = "Metric", values_to = "value")

# Save raw CSV data first
stats_csv_path <- file.path(output_dir, "allele_stats_by_group.csv")
write_csv(allele_stats_group, stats_csv_path)

cat(sprintf("Raw statistics saved to: %s\n", stats_csv_path))

# ==============================================================================
# 2. Format Table for Publication (GT Package)
# ==============================================================================

# Reshape to Wide Format for display (Metrics in rows, Groups in columns)
df_wide_pub <- allele_stats_group %>%
  pivot_wider(
    names_from = group,
    values_from = value
  ) %>%
  # Rename columns to match the target style
  rename(
    'Case Group' = case,
    'Control Group' = control
  )

# Translate/Format Metric names explicitly
# Using proper English scientific notation
df_wide_pub <- df_wide_pub %>%
  mutate(Metric = case_when(
    Metric == "allele1_est_mean"   ~ "Mean (Allele 1)",
    Metric == "allele1_est_median" ~ "Median (Allele 1)",
    Metric == "allele1_est_sd"     ~ "SD (Allele 1)",
    Metric == "allele1_est_iqr"    ~ "IQR (Allele 1)",
    
    Metric == "allele2_est_mean"   ~ "Mean (Allele 2)",
    Metric == "allele2_est_median" ~ "Median (Allele 2)",
    Metric == "allele2_est_sd"     ~ "SD (Allele 2)",
    Metric == "allele2_est_iqr"    ~ "IQR (Allele 2)",
    
    Metric == "n_obs"              ~ "Total Observations",
    TRUE ~ Metric # Keep others as is if any
  )) %>%
  # Custom Sort order (Optional: Obs -> Allele 1 -> Allele 2)
  arrange(factor(Metric, levels = c(
    "Total Observations",
    "Mean (Allele 1)", "Median (Allele 1)", "SD (Allele 1)", "IQR (Allele 1)",
    "Mean (Allele 2)", "Median (Allele 2)", "SD (Allele 2)", "IQR (Allele 2)"
  )))

# Create GT Table
tabela_gt <- gt(df_wide_pub) %>%
  # Number formatting
  fmt_number(
    columns = -Metric,      
    decimals = 2,
    use_seps = TRUE           
  ) %>%
  # Special formatting for Observation count (no decimals)
  fmt_number(
    columns = -Metric,
    rows = Metric == "Total Observations",
    decimals = 0
  ) %>%
  # Table Header
  tab_header(
    title = "Descriptive Statistics of STR Metrics",
    subtitle = "Comparison between Case and Control Groups"
  ) %>%
  # Column Labels
  cols_label(
    Metric = "Metric",
    `Case Group` = "Case Group",
    `Control Group` = "Control Group"
  ) %>%
  # Footnotes
  tab_footnote(
    footnote = "SD: Standard Deviation; IQR: Interquartile Range.",
    locations = cells_column_labels(columns = Metric)
  ) %>%
  # Style: Bold Metric names
  tab_style(
    style = cell_text(weight = "bold"),
    locations = cells_body(columns = Metric)
  )

# 3. Save as HTML
html_path <- file.path(output_dir, "table_summary_desc.html")
gtsave(tabela_gt, html_path)

cat(sprintf("Publication-ready table saved to: %s\n", html_path))


Raw statistics saved to: ./results/allele_stats_by_group.csv
Publication-ready table saved to: ./results/table_summary_desc.html


2. Calculate data distribution between loci and groups

In [36]:
# 0. Data Preparation
# Calculate mean_alleles
df_normality <- df_filtered_grouped %>%
  mutate(mean_alleles = (allele1_est + allele2_est) / 2)

# 1. Define Function to Run Shapiro Test Safely
safe_shapiro <- function(x) {
  # Remove NAs
  clean_x <- x[!is.na(x)]
  n <- length(clean_x)
  
  # Shapiro requires 3 to 5000 samples
  if (n < 3) {
    return(tibble(statistic = NA, p.value = NA, n_obs = n, status = "Too few samples"))
  }
  if (n > 5000) {
    # If a single locus has >5000 samples, we subsample to allow the test to run
    set.seed(123)
    clean_x <- sample(clean_x, 5000)
  }
  
  # Constant variance check (all values identical)
  if (var(clean_x) == 0) {
    return(tibble(statistic = NA, p.value = 1, n_obs = n, status = "Constant Variance"))
  }
  
  tryCatch({
    res <- shapiro_test(clean_x)
    return(tibble(statistic = res$statistic, p.value = res$p.value, n_obs = n, status = "OK"))
  }, error = function(e) return(tibble(statistic = NA, p.value = NA, n_obs = n, status = "Error")))
}

# 2. Run Normality Tests (Per Locus, Per Group, Per Allele/Variable)
normality_results <- df_normality %>%
  select(STRs_ID, group, allele1_est, allele2_est, mean_alleles) %>%
  pivot_longer(cols = c(allele1_est, allele2_est, mean_alleles), 
               names_to = "variable", 
               values_to = "value") %>%
  # VITAL: Grouping by STRs_ID prevents mixing different loci distributions
  group_by(STRs_ID, group, variable) %>%
  nest() %>%
  mutate(shapiro = map(data, ~safe_shapiro(.x$value))) %>%
  unnest(shapiro) %>%
  ungroup() %>%
  mutate(
    normality = case_when(
      is.na(p.value) ~ "Insufficient Data/Constant",
      p.value > 0.05 ~ "Normal",
      p.value <= 0.05 ~ "Non-Normal" # Significant deviation from normal curve
    )
  )

# 3. Summary of Results
cat("\n--- Normality Test Summary (Counts of Loci) ---\n")

# Count how many loci are Normal vs Non-Normal for each Variable/Group combination
summary_table <- normality_results %>%
  count(variable, group, normality) %>%
  pivot_wider(names_from = normality, values_from = n, values_fill = 0)

print(summary_table)

# 4. Save Results
output_dir <- "./results"
if (!dir.exists(output_dir)) dir.create(output_dir)

# Saving the detailed per-locus table
write_csv(normality_results, file.path(output_dir, "shapiro_normality_per_locus_allele_group.csv"))

cat(sprintf("\nDetailed results saved to: %s/shapiro_normality_per_locus_allele_group.csv\n", output_dir))



--- Normality Test Summary (Counts of Loci) ---
# A tibble: 6 × 5
  variable     group   `Insufficient Data/Constant` `Non-Normal` Normal
  <chr>        <chr>                          <int>        <int>  <int>
1 allele1_est  case                             530         1364   1583
2 allele1_est  control                          242         2332   1396
3 allele2_est  case                             530         1588   1359
4 allele2_est  control                          242         2961    767
5 mean_alleles case                             530         1545   1402
6 mean_alleles control                          242         2744    984

Detailed results saved to: ./results/shapiro_normality_per_locus_allele_group.csv


3. Mann-whitney analysis between the major allele length

In [37]:
# =======================================================
# 0. DATA PREPARATION & DEDUPLICATION (STEP ZERO)
# =======================================================
cat("--- Starting Data Preparation (Target: Allele 2) ---\n")

# 1. Create Identifiers and Select Metric
df_prepared <- df_filtered_grouped %>%
  # Create the Locus ID
  mutate(locus_definition = str_remove(STRs_ID, ":[^:]+$")) %>%
  
  # DEFINING THE METRIC OF INTEREST: ALLELE 2
  mutate(target_metric = allele2_est)

# 2. Deduplication (Ensure Uniqueness: 1 row per Sample per Locus)
df_unique_samples <- df_prepared %>%
  group_by(locus_definition, sample_id, group) %>%
  summarise(
    # Mean of the target metric (in case of technical replicates)
    target_metric = mean(target_metric, na.rm = TRUE),
    
    # Preserve metadata
    gene_name = first(gene_name),
    chrom     = first(chrom),
    start     = first(start),
    region    = first(region),   
    STRs_ID   = first(STRs_ID),  
    
    .groups = "drop"
  )

# Update main dataframe
df_per_locus <- df_unique_samples

cat("Data prepared. Deduplicated rows:", nrow(df_per_locus), "\n")
cat("Using metric: Allele 2\n")


# =======================================================
# 1. FILTER ELIGIBLE LOCI
# =======================================================
loci_counts <- df_per_locus %>%
  group_by(locus_definition, group) %>%
  summarise(n = sum(!is.na(target_metric)), .groups = "drop") %>%
  pivot_wider(names_from = group, values_from = n, values_fill = 0)

# Filter loci with enough samples
loci_to_test <- loci_counts %>%
  filter(case >= 5 & control >= 5) %>%
  pull(locus_definition)

cat(sprintf("Eligible loci for testing: %d\n", length(loci_to_test)))


# =======================================================
# 2. METADATA LOOKUP TABLE
# =======================================================
cat("\n--- Creating Metadata Dictionary ---\n")

loci_metadata <- df_per_locus %>%
  # <--- ADICIONADO: STRs_ID no select
  select(locus_definition, gene_name, chrom, start, region, STRs_ID) %>%
  group_by(locus_definition) %>%
  summarise(
    gene_name = first(na.omit(gene_name)),
    chrom     = first(na.omit(chrom)),
    start     = first(na.omit(start)),
    region    = first(na.omit(region)),
    STRs_ID   = first(na.omit(STRs_ID)), 
    n_versions = n_distinct(gene_name, na.rm = TRUE),
    .groups = "drop"
  )

if (any(loci_metadata$n_versions > 1)) {
  warning("Some loci have multiple gene names. Using the first valid occurrence.")
}
loci_metadata <- loci_metadata %>% select(-n_versions)


# =======================================================
# 3. SAFE STATISTICAL FUNCTION
# =======================================================
safe_wilcox_metric <- function(data) {
  tryCatch({
    # Check for variance in target_metric
    if (n_distinct(data$target_metric) < 2) return(NULL)
    
    # Mann-Whitney test on target_metric
    test_res <- data %>% wilcox_test(target_metric ~ group)
    
    # Stats Summary
    stats_summary <- data %>%
      group_by(group) %>%
      summarise(
        median_val = median(target_metric, na.rm = TRUE),
        iqr_val    = IQR(target_metric, na.rm = TRUE),
        .groups = "drop"
      ) %>%
      pivot_wider(
        names_from = group,
        values_from = c(median_val, iqr_val),
        names_glue = "{.value}_{group}"
      )
    
    return(bind_cols(test_res, stats_summary))
  }, error = function(e) { return(NULL) })
}


# =======================================================
# 4. RUN TESTS
# =======================================================
cat("\n--- Running Statistical Tests ---\n")

nested_results <- df_per_locus %>%
  filter(locus_definition %in% loci_to_test) %>%
  
  # Select metric of interest
  select(locus_definition, sample_id, group, target_metric) %>%
  
  group_by(locus_definition) %>%
  nest() %>%
  
  mutate(model = map(data, safe_wilcox_metric)) %>%
  
  filter(!map_lgl(model, is.null))


# =======================================================
# 5. METADATA REINCORPORATION
# =======================================================
cat("\n--- Consolidating Results ---\n")

# Statistical Results
results_mw_allele2 <- nested_results %>%
  select(-data) %>%
  unnest(model) %>%
  ungroup() %>%
  left_join(loci_metadata, by = "locus_definition") %>%
  select(locus_definition, STRs_ID, gene_name, chrom, start, region, everything()) %>%
  adjust_pvalue(method = "BH") %>%
  add_significance("p.adj") %>%
  arrange(p)

# Raw Observations
raw_obs_allele2 <- nested_results %>%
  ungroup() %>%
  select(locus_definition, data) %>%
  unnest(data) %>%
  left_join(loci_metadata, by = "locus_definition") %>%
  select(locus_definition, STRs_ID, gene_name, sample_id, group, target_metric)


# =======================================================
# 6. OUTPUT
# =======================================================
cat("\n=== FINAL SUMMARY (Allele 2) ===\n")
cat("Loci analyzed:", nrow(results_mw_allele2), "\n")
cat("Significant (p.adj < 0.05):", sum(results_mw_allele2$p.adj < 0.05, na.rm = TRUE), "\n")

print(head(results_mw_allele2))

# Save with specific names
if(!dir.exists("results")) dir.create("results")
write_csv(results_mw_allele2, "results/allele2_stats_results_mann-whitney.csv")
write_csv(raw_obs_allele2, "results/allele2_raw_observations_mann-whitney.csv")


--- Starting Data Preparation (Target: Allele 2) ---
Data prepared. Deduplicated rows: 144796 
Using metric: Allele 2
Eligible loci for testing: 1922

--- Creating Metadata Dictionary ---

--- Running Statistical Tests ---

--- Consolidating Results ---

=== FINAL SUMMARY (Allele 2) ===
Loci analyzed: 1922 
Significant (p.adj < 0.05): 0 
# A tibble: 6 × 19
  locus_definition  STRs_ID    gene_name chrom  start region .y.   group1 group2
  <chr>             <chr>      <chr>     <chr>  <dbl> <chr>  <chr> <chr>  <chr> 
1 chr1:8018387:C    chr1:8018… ERRFI1    1     8.02e6 intron targ… case   contr…
2 chr13:91965100:C  chr13:919… GPC5      13    9.20e7 intron targ… case   contr…
3 chr15:38983802:C  chr15:389… AC013652… 15    3.90e7 intron targ… case   contr…
4 chr6:156000320:C  chr6:1560… AL589693… 6     1.56e8 intron targ… case   contr…
5 chr3:169212653:AT chr3:1692… MECOM     3     1.69e8 intron targ… case   contr…
6 chr17:57467841:G  chr17:574… MSI2      17    5.75e7 intron targ… case   

4. Mann-whitney analysis between the mean length of alleles

In [38]:
# =======================================================
# ANALYSIS: MEAN ALLELES NO-OVERLAP DETECTION (CASE vs CONTROL)
# =======================================================
cat("--- Starting No-Overlap Analysis for Mean Alleles ---\n")


# =======================================================
# 1. PREPARE DATA FOR NO-OVERLAP ANALYSIS
# =======================================================

# Force recreation with the correct metric (Mean Alleles)
cat("--- Preparing data with Mean Alleles metric ---\n")

df_prepared <- df_filtered_grouped %>%
  mutate(locus_definition = str_remove(STRs_ID, ":[^:]+$")) %>%
  mutate(mean_alleles_val = (allele1_est + allele2_est) / 2)

df_per_locus <- df_prepared %>%
  group_by(locus_definition, sample_id, group) %>%
  summarise(
    mean_alleles_val = mean(mean_alleles_val, na.rm = TRUE),
    gene_name = first(gene_name),
    chrom     = first(chrom),
    start     = first(start),
    region    = first(region),
    STRs_ID   = first(STRs_ID),
    .groups = "drop"
  )

cat("Data prepared. Rows:", nrow(df_per_locus), "\n")


# =======================================================
# 2. CALCULATE STATISTICS PER LOCUS AND GROUP
# =======================================================
cat("\n--- Computing Statistics per Locus ---\n")

locus_stats <- df_per_locus %>%
  filter(!is.na(mean_alleles_val)) %>%
  group_by(locus_definition, group) %>%
  summarise(
    median_mean = median(mean_alleles_val, na.rm = TRUE),
    min_mean    = min(mean_alleles_val, na.rm = TRUE),
    max_mean    = max(mean_alleles_val, na.rm = TRUE),
    mean_value  = mean(mean_alleles_val, na.rm = TRUE),
    sd_mean     = sd(mean_alleles_val, na.rm = TRUE),
    n_samples   = n(),
    .groups = "drop"
  )

# Display the structure
cat("Locus stats calculated. Rows:", nrow(locus_stats), "\n")
print(head(locus_stats))


# =======================================================
# 3. IDENTIFY NO-OVERLAP VARIANTS
# =======================================================
cat("\n--- Identifying No-Overlap Variants ---\n")

# Pivot to compare case vs control side by side
locus_comparison <- locus_stats %>%
  pivot_wider(
    id_cols = locus_definition,
    names_from = group,
    values_from = c(median_mean, min_mean, max_mean, mean_value, sd_mean, n_samples),
    names_glue = "{.value}_{group}"
  )

# Detect no-overlap WITH MINIMUM SAMPLE SIZE REQUIREMENT
# Only consider variants with at least N samples in BOTH groups
MIN_SAMPLES_PER_GROUP <- 3  # Adjust this threshold as needed

locus_comparison <- locus_comparison %>%
  mutate(
    has_no_overlap = (n_samples_case >= MIN_SAMPLES_PER_GROUP & 
                      n_samples_control >= MIN_SAMPLES_PER_GROUP) &
                     ((min_mean_case > max_mean_control) | 
                      (min_mean_control > max_mean_case)),
    
    overlap_type = case_when(
      n_samples_case < MIN_SAMPLES_PER_GROUP ~ "insufficient_case_samples",
      n_samples_control < MIN_SAMPLES_PER_GROUP ~ "insufficient_control_samples",
      min_mean_case > max_mean_control ~ "case_higher",
      min_mean_control > max_mean_case ~ "control_higher",
      TRUE ~ "overlap"
    )
  )

# Recover metadata
locus_with_metadata <- locus_comparison %>%
  left_join(
    df_per_locus %>% 
      select(locus_definition, gene_name, region, STRs_ID, chrom, start) %>%
      distinct(),
    by = "locus_definition"
  )

cat("Total loci:", nrow(locus_with_metadata), "\n")
cat("Loci with NO-OVERLAP (min", MIN_SAMPLES_PER_GROUP, "samples per group):", 
    sum(locus_with_metadata$has_no_overlap, na.rm = TRUE), "\n")
cat("\n")


# =======================================================
# 4. CREATE OUTPUT CSV 1: DETAILED STATISTICS PER LOCUS
# =======================================================
cat("--- Creating Statistics CSV ---\n")

csv_statistics <- locus_with_metadata %>%
  select(
    locus_definition,
    STRs_ID,
    gene_name,
    region,
    chrom,
    start,
    
    # Case group
    median_mean_case,
    min_mean_case,
    max_mean_case,
    mean_value_case,
    sd_mean_case,
    n_samples_case,
    
    # Control group
    median_mean_control,
    min_mean_control,
    max_mean_control,
    mean_value_control,
    sd_mean_control,
    n_samples_control,
    
    # Overlap info
    has_no_overlap,
    overlap_type
  ) %>%
  arrange(desc(has_no_overlap), locus_definition)

print(head(csv_statistics, 10))

# Create results directory if it doesn't exist
if(!dir.exists("results")) dir.create("results")

write_csv(
  csv_statistics,
  "results/mean_alleles_no_overlap_statistics_per_locus.csv"
)

cat("✓ Saved: mean_alleles_no_overlap_statistics_per_locus.csv\n")


# =======================================================
# 5. CREATE OUTPUT CSV 2: ONLY NO-OVERLAP VARIANTS
# =======================================================
cat("\n--- Creating No-Overlap Variants CSV ---\n")

csv_no_overlap_only <- locus_with_metadata %>%
  filter(has_no_overlap == TRUE) %>%
  select(
    locus_definition,
    STRs_ID,
    gene_name,
    region,
    chrom,
    start,
    
    # Case group
    median_mean_case,
    min_mean_case,
    max_mean_case,
    n_samples_case,
    
    # Control group
    median_mean_control,
    min_mean_control,
    max_mean_control,
    n_samples_control,
    
    # Interpretation
    overlap_type
  ) %>%
  arrange(overlap_type, locus_definition)

print(csv_no_overlap_only)

write_csv(
  csv_no_overlap_only,
  "results/mean_alleles_no_overlap_variants_only.csv"
)

cat("✓ Saved: mean_alleles_no_overlap_variants_only.csv\n")


# =======================================================
# 6. SUMMARY REPORT
# =======================================================
cat("\n=== NO-OVERLAP ANALYSIS SUMMARY (Mean Alleles) ===\n")
cat("Total loci analyzed:", nrow(csv_statistics), "\n")
cat("Loci with NO-OVERLAP (min", MIN_SAMPLES_PER_GROUP, "samples per group):", 
    nrow(csv_no_overlap_only), "\n")
cat("\nNo-overlap breakdown:\n")

if(nrow(csv_no_overlap_only) > 0) {
  breakdown <- csv_no_overlap_only %>%
    group_by(overlap_type) %>%
    summarise(
      count = n(),
      .groups = "drop"
    )
  
  print(breakdown)
} else {
  cat("No variants without overlap found.\n")
}

--- Starting No-Overlap Analysis for Mean Alleles ---
--- Preparing data with Mean Alleles metric ---
Data prepared. Rows: 144796 

--- Computing Statistics per Locus ---
Locus stats calculated. Rows: 7447 
# A tibble: 6 × 8
  locus_definition  group   median_mean min_mean max_mean mean_value sd_mean
  <chr>             <chr>         <dbl>    <dbl>    <dbl>      <dbl>   <dbl>
1 chr10:10857639:AT case           8.43     5.78     12.2       8.91    2.13
2 chr10:10857639:AT control        9.49     2.86     19.0      10.1     3.34
3 chr10:10994724:C  case          20.7     14.8      31.0      21.1     5.09
4 chr10:10994724:C  control       17.1     13.5      30.5      18.7     3.84
5 chr10:11005244:AG case           8.13     5.96     14.6       8.75    2.78
6 chr10:11005244:AG control       10.1      2.33     18.5       9.81    3.55
# ℹ 1 more variable: n_samples <int>

--- Identifying No-Overlap Variants ---
Total loci: 4046 
Loci with NO-OVERLAP (min 3 samples per group): 2 

--- Creatin

5. Identification of loci without overlap in relation of the major allele

In [39]:
# =======================================================
# ANALYSIS: ALLELE 2 NO-OVERLAP DETECTION (CASE vs CONTROL)
# =======================================================
cat("--- Starting No-Overlap Analysis for Allele 2 ---\n")


# =======================================================
# 1. PREPARE DATA FOR NO-OVERLAP ANALYSIS
# =======================================================

# Force recreation with the correct metric (Allele 2)
cat("--- Preparing data with Allele 2 metric ---\n")

df_prepared <- df_filtered_grouped %>%
  mutate(locus_definition = str_remove(STRs_ID, ":[^:]+$")) %>%
  mutate(allele2_val = allele2_est)

df_per_locus <- df_prepared %>%
  group_by(locus_definition, sample_id, group) %>%
  summarise(
    allele2_val = mean(allele2_val, na.rm = TRUE),
    gene_name = first(gene_name),
    chrom     = first(chrom),
    start     = first(start),
    region    = first(region),
    STRs_ID   = first(STRs_ID),
    .groups = "drop"
  )

cat("Data prepared. Rows:", nrow(df_per_locus), "\n")


# =======================================================
# 2. CALCULATE STATISTICS PER LOCUS AND GROUP
# =======================================================
cat("\n--- Computing Statistics per Locus ---\n")

locus_stats <- df_per_locus %>%
  filter(!is.na(allele2_val)) %>%
  group_by(locus_definition, group) %>%
  summarise(
    median_allele2 = median(allele2_val, na.rm = TRUE),
    min_allele2    = min(allele2_val, na.rm = TRUE),
    max_allele2    = max(allele2_val, na.rm = TRUE),
    mean_allele2   = mean(allele2_val, na.rm = TRUE),
    sd_allele2     = sd(allele2_val, na.rm = TRUE),
    n_samples      = n(),
    .groups = "drop"
  )

# Display the structure
cat("Locus stats calculated. Rows:", nrow(locus_stats), "\n")
print(head(locus_stats))


# =======================================================
# 3. IDENTIFY NO-OVERLAP VARIANTS
# =======================================================
cat("\n--- Identifying No-Overlap Variants ---\n")

# Pivot to compare case vs control side by side
locus_comparison <- locus_stats %>%
  pivot_wider(
    id_cols = locus_definition,
    names_from = group,
    values_from = c(median_allele2, min_allele2, max_allele2, mean_allele2, sd_allele2, n_samples),
    names_glue = "{.value}_{group}"
  )

# Detect no-overlap:
# - If min_allele2_case > max_allele2_control: case all greater
# - If min_allele2_control > max_allele2_case: control all greater
MIN_SAMPLES_PER_GROUP <- 3

locus_comparison <- locus_comparison %>%
  mutate(
    has_no_overlap = (n_samples_case >= MIN_SAMPLES_PER_GROUP & 
                      n_samples_control >= MIN_SAMPLES_PER_GROUP) &
                     ((min_allele2_case > max_allele2_control) | 
                      (min_allele2_control > max_allele2_case)),
    
    overlap_type = case_when(
      n_samples_case < MIN_SAMPLES_PER_GROUP ~ "insufficient_case_samples",
      n_samples_control < MIN_SAMPLES_PER_GROUP ~ "insufficient_control_samples",
      min_allele2_case > max_allele2_control ~ "case_higher",
      min_allele2_control > max_allele2_case ~ "control_higher",
      TRUE ~ "overlap"
    )
  )

# Recover metadata
locus_with_metadata <- locus_comparison %>%
  left_join(
    df_per_locus %>% 
      select(locus_definition, gene_name, region, STRs_ID, chrom, start) %>%
      distinct(),
    by = "locus_definition"
  )

cat("Total loci:", nrow(locus_with_metadata), "\n")
cat("Loci with NO-OVERLAP (min", MIN_SAMPLES_PER_GROUP, "samples per group):", 
    sum(locus_with_metadata$has_no_overlap, na.rm = TRUE), "\n")
cat("\n")
cat("Breakdown by category:\n")
cat("  case_higher:", sum(locus_with_metadata$overlap_type == "case_higher", na.rm = TRUE), "\n")
cat("  control_higher:", sum(locus_with_metadata$overlap_type == "control_higher", na.rm = TRUE), "\n")
cat("  insufficient_case_samples:", sum(locus_with_metadata$overlap_type == "insufficient_case_samples", na.rm = TRUE), "\n")
cat("  insufficient_control_samples:", sum(locus_with_metadata$overlap_type == "insufficient_control_samples", na.rm = TRUE), "\n")
cat("  overlap:", sum(locus_with_metadata$overlap_type == "overlap", na.rm = TRUE), "\n")
cat("\n")


# =======================================================
# 4. CREATE OUTPUT CSV 1: DETAILED STATISTICS PER LOCUS
# =======================================================
cat("--- Creating Statistics CSV ---\n")

csv_statistics <- locus_with_metadata %>%
  select(
    locus_definition,
    STRs_ID,
    gene_name,
    region,
    chrom,
    start,
    
    # Case group
    median_allele2_case,
    min_allele2_case,
    max_allele2_case,
    mean_allele2_case,
    sd_allele2_case,
    n_samples_case,
    
    # Control group
    median_allele2_control,
    min_allele2_control,
    max_allele2_control,
    mean_allele2_control,
    sd_allele2_control,
    n_samples_control,
    
    # Overlap info
    has_no_overlap,
    overlap_type
  ) %>%
  arrange(desc(has_no_overlap), locus_definition)

print(head(csv_statistics, 10))

# Create results directory if it doesn't exist
if(!dir.exists("results")) dir.create("results")

write_csv(
  csv_statistics,
  "results/allele2_no_overlap_statistics_per_locus.csv"
)

cat("✓ Saved: allele2_no_overlap_statistics_per_locus.csv\n")


# =======================================================
# 5. CREATE OUTPUT CSV 2: ONLY NO-OVERLAP VARIANTS
# =======================================================
cat("\n--- Creating No-Overlap Variants CSV ---\n")

csv_no_overlap_only <- locus_with_metadata %>%
  filter(has_no_overlap == TRUE) %>%
  select(
    locus_definition,
    STRs_ID,
    gene_name,
    region,
    chrom,
    start,
    
    # Case group
    median_allele2_case,
    min_allele2_case,
    max_allele2_case,
    n_samples_case,
    
    # Control group
    median_allele2_control,
    min_allele2_control,
    max_allele2_control,
    n_samples_control,
    
    # Interpretation
    overlap_type
  ) %>%
  arrange(overlap_type, locus_definition)

print(csv_no_overlap_only)

write_csv(
  csv_no_overlap_only,
  "results/allele2_no_overlap_variants_only.csv"
)

cat("✓ Saved: allele2_no_overlap_variants_only.csv\n")


# =======================================================
# 6. SUMMARY REPORT
# =======================================================
cat("\n=== NO-OVERLAP ANALYSIS SUMMARY (Allele 2) ===\n")
cat("Total loci analyzed:", nrow(csv_statistics), "\n")
cat("Loci with NO-OVERLAP:", nrow(csv_no_overlap_only), "\n")
cat("\nNo-overlap breakdown:\n")

if(nrow(csv_no_overlap_only) > 0) {
  breakdown <- csv_no_overlap_only %>%
    group_by(overlap_type) %>%
    summarise(
      count = n(),
      .groups = "drop"
    )
  
  print(breakdown)
} else {
  cat("No variants without overlap found.\n")
}

--- Starting No-Overlap Analysis for Allele 2 ---
--- Preparing data with Allele 2 metric ---
Data prepared. Rows: 144796 

--- Computing Statistics per Locus ---
Locus stats calculated. Rows: 7447 
# A tibble: 6 × 8
  locus_definition  group   median_allele2 min_allele2 max_allele2 mean_allele2
  <chr>             <chr>            <dbl>       <dbl>       <dbl>        <dbl>
1 chr10:10857639:AT case              16.8        14          21.4         17.8
2 chr10:10857639:AT control           17.7        10.7        41.4         19.3
3 chr10:10994724:C  case              41.4        29.6        62.0         42.3
4 chr10:10994724:C  control           34.2        27.0        61.0         37.4
5 chr10:11005244:AG case              16.8        13.4        29.2         18.0
6 chr10:11005244:AG control           19.2        10.5        37.0         20.2
# ℹ 2 more variables: sd_allele2 <dbl>, n_samples <int>

--- Identifying No-Overlap Variants ---
Total loci: 4046 
Loci with NO-OVERLAP (min 3 

6. Identification of loci without overlap in relation of mean alleles

In [40]:
# =======================================================
# ANALYSIS: MEAN ALLELES NO-OVERLAP DETECTION (CASE vs CONTROL)
# =======================================================
cat("--- Starting No-Overlap Analysis for Mean Alleles ---\n")


# =======================================================
# 1. PREPARE DATA FOR NO-OVERLAP ANALYSIS
# =======================================================

# Force recreation with the correct metric (Mean Alleles)
cat("--- Preparing data with Mean Alleles metric ---\n")

df_prepared <- df_filtered_grouped %>%
  mutate(locus_definition = str_remove(STRs_ID, ":[^:]+$")) %>%
  mutate(mean_alleles_val = (allele1_est + allele2_est) / 2)

df_per_locus <- df_prepared %>%
  group_by(locus_definition, sample_id, group) %>%
  summarise(
    mean_alleles_val = mean(mean_alleles_val, na.rm = TRUE),
    gene_name = first(gene_name),
    chrom     = first(chrom),
    start     = first(start),
    region    = first(region),
    STRs_ID   = first(STRs_ID),
    .groups = "drop"
  )

cat("Data prepared. Rows:", nrow(df_per_locus), "\n")


# =======================================================
# 2. CALCULATE STATISTICS PER LOCUS AND GROUP
# =======================================================
cat("\n--- Computing Statistics per Locus ---\n")

locus_stats <- df_per_locus %>%
  filter(!is.na(mean_alleles_val)) %>%
  group_by(locus_definition, group) %>%
  summarise(
    median_mean = median(mean_alleles_val, na.rm = TRUE),
    min_mean    = min(mean_alleles_val, na.rm = TRUE),
    max_mean    = max(mean_alleles_val, na.rm = TRUE),
    mean_value  = mean(mean_alleles_val, na.rm = TRUE),
    sd_mean     = sd(mean_alleles_val, na.rm = TRUE),
    n_samples   = n(),
    .groups = "drop"
  )

# Display the structure
cat("Locus stats calculated. Rows:", nrow(locus_stats), "\n")
print(head(locus_stats))


# =======================================================
# 3. IDENTIFY NO-OVERLAP VARIANTS
# =======================================================
cat("\n--- Identifying No-Overlap Variants ---\n")

# Pivot to compare case vs control side by side
locus_comparison <- locus_stats %>%
  pivot_wider(
    id_cols = locus_definition,
    names_from = group,
    values_from = c(median_mean, min_mean, max_mean, mean_value, sd_mean, n_samples),
    names_glue = "{.value}_{group}"
  )

# Detect no-overlap WITH MINIMUM SAMPLE SIZE REQUIREMENT
# Only consider variants with at least N samples in BOTH groups
MIN_SAMPLES_PER_GROUP <- 3  # Adjust this threshold as needed

locus_comparison <- locus_comparison %>%
  mutate(
    has_no_overlap = (n_samples_case >= MIN_SAMPLES_PER_GROUP & 
                      n_samples_control >= MIN_SAMPLES_PER_GROUP) &
                     ((min_mean_case > max_mean_control) | 
                      (min_mean_control > max_mean_case)),
    
    overlap_type = case_when(
      n_samples_case < MIN_SAMPLES_PER_GROUP ~ "insufficient_case_samples",
      n_samples_control < MIN_SAMPLES_PER_GROUP ~ "insufficient_control_samples",
      min_mean_case > max_mean_control ~ "case_higher",
      min_mean_control > max_mean_case ~ "control_higher",
      TRUE ~ "overlap"
    )
  )

# Recover metadata
locus_with_metadata <- locus_comparison %>%
  left_join(
    df_per_locus %>% 
      select(locus_definition, gene_name, region, STRs_ID, chrom, start) %>%
      distinct(),
    by = "locus_definition"
  )

cat("Total loci:", nrow(locus_with_metadata), "\n")
cat("Loci with NO-OVERLAP (min", MIN_SAMPLES_PER_GROUP, "samples per group):", 
    sum(locus_with_metadata$has_no_overlap, na.rm = TRUE), "\n")
cat("\n")


# =======================================================
# 4. CREATE OUTPUT CSV 1: DETAILED STATISTICS PER LOCUS
# =======================================================
cat("--- Creating Statistics CSV ---\n")

csv_statistics <- locus_with_metadata %>%
  select(
    locus_definition,
    STRs_ID,
    gene_name,
    region,
    chrom,
    start,
    
    # Case group
    median_mean_case,
    min_mean_case,
    max_mean_case,
    mean_value_case,
    sd_mean_case,
    n_samples_case,
    
    # Control group
    median_mean_control,
    min_mean_control,
    max_mean_control,
    mean_value_control,
    sd_mean_control,
    n_samples_control,
    
    # Overlap info
    has_no_overlap,
    overlap_type
  ) %>%
  arrange(desc(has_no_overlap), locus_definition)

print(head(csv_statistics, 10))

# Create results directory if it doesn't exist
if(!dir.exists("results")) dir.create("results")

write_csv(
  csv_statistics,
  "results/mean_alleles_no_overlap_statistics_per_locus.csv"
)

cat("✓ Saved: mean_alleles_no_overlap_statistics_per_locus.csv\n")


# =======================================================
# 5. CREATE OUTPUT CSV 2: ONLY NO-OVERLAP VARIANTS
# =======================================================
cat("\n--- Creating No-Overlap Variants CSV ---\n")

csv_no_overlap_only <- locus_with_metadata %>%
  filter(has_no_overlap == TRUE) %>%
  select(
    locus_definition,
    STRs_ID,
    gene_name,
    region,
    chrom,
    start,
    
    # Case group
    median_mean_case,
    min_mean_case,
    max_mean_case,
    n_samples_case,
    
    # Control group
    median_mean_control,
    min_mean_control,
    max_mean_control,
    n_samples_control,
    
    # Interpretation
    overlap_type
  ) %>%
  arrange(overlap_type, locus_definition)

print(csv_no_overlap_only)

write_csv(
  csv_no_overlap_only,
  "results/mean_alleles_no_overlap_variants_only.csv"
)

cat("✓ Saved: mean_alleles_no_overlap_variants_only.csv\n")


# =======================================================
# 6. SUMMARY REPORT
# =======================================================
cat("\n=== NO-OVERLAP ANALYSIS SUMMARY (Mean Alleles) ===\n")
cat("Total loci analyzed:", nrow(csv_statistics), "\n")
cat("Loci with NO-OVERLAP (min", MIN_SAMPLES_PER_GROUP, "samples per group):", 
    nrow(csv_no_overlap_only), "\n")
cat("\nNo-overlap breakdown:\n")

if(nrow(csv_no_overlap_only) > 0) {
  breakdown <- csv_no_overlap_only %>%
    group_by(overlap_type) %>%
    summarise(
      count = n(),
      .groups = "drop"
    )
  
  print(breakdown)
} else {
  cat("No variants without overlap found.\n")
}

--- Starting No-Overlap Analysis for Mean Alleles ---
--- Preparing data with Mean Alleles metric ---


Data prepared. Rows: 144796 

--- Computing Statistics per Locus ---
Locus stats calculated. Rows: 7447 
# A tibble: 6 × 8
  locus_definition  group   median_mean min_mean max_mean mean_value sd_mean
  <chr>             <chr>         <dbl>    <dbl>    <dbl>      <dbl>   <dbl>
1 chr10:10857639:AT case           8.43     5.78     12.2       8.91    2.13
2 chr10:10857639:AT control        9.49     2.86     19.0      10.1     3.34
3 chr10:10994724:C  case          20.7     14.8      31.0      21.1     5.09
4 chr10:10994724:C  control       17.1     13.5      30.5      18.7     3.84
5 chr10:11005244:AG case           8.13     5.96     14.6       8.75    2.78
6 chr10:11005244:AG control       10.1      2.33     18.5       9.81    3.55
# ℹ 1 more variable: n_samples <int>

--- Identifying No-Overlap Variants ---
Total loci: 4046 
Loci with NO-OVERLAP (min 3 samples per group): 2 

--- Creating Statistics CSV ---
# A tibble: 10 × 20
   locus_definition  STRs_ID      gene_name region chrom  sta